In [14]:
import pandas as pd
import numpy as np
import re

In [15]:
df = pd.read_csv("../data/raw/riyadh_resturants_raw.csv")
df_clean = df.copy()

In [16]:
df_clean["lat"] = pd.to_numeric(df_clean["lat"] , errors="coerce")
df_clean["lng"] = pd.to_numeric(df_clean["lng"] , errors="coerce")

In [17]:
df_clean = df_clean.drop_duplicates()

In [18]:
df_clean = df_clean.drop_duplicates(subset=["name" , "lat" , "lng"])

df_clean["price"].value_counts()

price
Cheap             13495
Moderate           3980
Expensive           303
Very Expensive       63
Name: count, dtype: int64

In [19]:
df_clean["price"] = df_clean["price"].str.strip().str.lower()
df_clean["price"] = df_clean["price"].replace("nan", np.nan)

price_mode = df["price"].mode()[0]
price_mode = price_mode.strip().lower()
df_clean["price"] = df_clean["price"].fillna(price_mode)


price_map = {
    "cheap": 0,
    "moderate": 1,
    "expensive": 2,
    "very expensive": 3,
}

df_clean["price_code"] = df_clean["price"].map(price_map)


In [20]:
rating_mean = df_clean["rating"].mean()
df_clean["rating"] = df_clean["rating"].fillna(rating_mean)

In [21]:
for col in ["name" , "categories" , "address"]:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].str.strip()

df_clean["name"] = df_clean["name"].replace(r"[$']" , "" , regex=True)

In [22]:
df_clean.likes = df_clean.likes.fillna(0)

In [23]:
def extract_postcode(addr):
    if pd.isna(addr):
        return np.nan
    addr = str(addr)
    matches = re.findall(r"\b(\d{5})\b", addr)
    if not matches:
        return np.nan
    return matches[-1]

df_clean["postcode"] = df_clean["address"].apply(extract_postcode)

In [24]:
df_clean.to_csv("../data/clean/riyadh_restaurants_clean.csv", index=False)